In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkConf
import json
from pyspark import SparkContext,SparkFiles
import numpy as np

#matchInfoDF
#rankingOfLandingPlayerDF
#rankingOfDistanceCircleDF
#rankingOfcombatCountDF
#rankingOfAttackDistanceDF
#rankingOfcombatDistanceDF 큰차이없음
#rankingOfMoveDistanceDF 큰차이없음

In [2]:
matchDF = spark.read.format("json").load("/sparkdata/PUBG/MatchesList1.json").distinct()
matchDF.printSchema()

matchDF = matchDF.select('id','createdAt','mapName','duration')  \
    .filter((col("isCustomMatch")==False) & (col("matchType")=="competitive"))
            
matchDF.show(5)

root
 |-- createdAt: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- gameMode: string (nullable = true)
 |-- id: string (nullable = true)
 |-- isCustomMatch: boolean (nullable = true)
 |-- mapName: string (nullable = true)
 |-- matchType: string (nullable = true)
 |-- seasonState: string (nullable = true)
 |-- shardId: string (nullable = true)
 |-- stats: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- titleId: string (nullable = true)
 |-- type: string (nullable = true)

+--------------------+--------------------+-----------+--------+
|                  id|           createdAt|    mapName|duration|
+--------------------+--------------------+-----------+--------+
|57dff48b-45a6-4e2...|2022-05-26T01:59:10Z|Baltic_Main|    1844|
|17562fa2-00f7-4dd...|2022-05-15T22:48:17Z|Baltic_Main|    1881|
|33775097-2850-429...|2022-05-20T20:55:55Z| Tiger_Main|    1897|
|ad7decd7-7fb4-405...|2022-05-20T01:36:18Z|Desert_Main|    1692|
|5d3d4b8a-121e-4dd...|2022-05-

In [3]:
circlePositionDF = spark.read.format("json").load("/sparkdata/PUBG/Matches1_CirclePosition.json").distinct()
circlePositionDF.printSchema()
circlePositionDF.show(5)

root
 |-- blue: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- matchId: string (nullable = true)
 |-- red: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- white: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)

+--------------------+--------------------+--------------------+--------------------+
|                blue|             matchId|                 red|               white|
+--------------------+--------------------+--------------------+--------------------+
|[[0.0, 406387.5, ...|88951693-d6cc-43c...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|
|[[0.0, 406387.5, ...|7139c667-0799-4f1...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|
|[[0.0, 406387.5, ...|4fb5d5e5-11cd-4c0...|[[0.0, 0.0, 0.0, ...|[[0.0, 0.0, 0.0, ...|
|[[0.0, 408000.0, ...|0c4c5491-958

In [4]:
airplaneRouteDF = spark.read.format("json").load("/sparkdata/PUBG/Matches1_AirPlaneRoute.json").distinct()
airplaneRouteDF = airplaneRouteDF.na.drop()
airplaneRouteDF.printSchema()
airplaneRouteDF.show(5)

root
 |-- intercept: double (nullable = true)
 |-- mapX: long (nullable = true)
 |-- matchId: string (nullable = true)
 |-- slope: double (nullable = true)

+------------------+------+--------------------+-------------+
|         intercept|  mapX|             matchId|        slope|
+------------------+------+--------------------+-------------+
|-1033813.819757835|409600|ad8cab68-412a-436...| 9.7036999784|
|  42688.5710657231|204800|d370e4c8-41ab-455...| 0.1880605381|
| -47814.6373893977|819200|f040f45c-c6ce-465...| 1.4857299105|
| 440102.2867084884|819200|591d0213-56f6-4fc...| 0.2204598748|
|-8539082.767434431|819200|a0639006-045e-42c...|21.0765695416|
+------------------+------+--------------------+-------------+
only showing top 5 rows



In [5]:
matchInfoDF = matchDF.join(airplaneRouteDF,matchDF.id==airplaneRouteDF.matchId,"inner") \
    .join(circlePositionDF,matchDF.id==circlePositionDF.matchId,"inner") \
    .select(matchDF.id.alias("matchId"),matchDF.createdAt,matchDF.mapName,matchDF.duration \
            , airplaneRouteDF.mapX.alias("mapSideLength"), airplaneRouteDF.intercept.alias("airplaneRoute_intercept") \
            , airplaneRouteDF.slope.alias("airplaneRoute_slope") \
            , circlePositionDF.blue.alias("blueCircle"),circlePositionDF.white.alias("whiteCircle"))

matchInfoDF.show(10)
matchInfoDF.printSchema()

+--------------------+--------------------+-----------+--------+-------------+-----------------------+-------------------+--------------------+--------------------+
|             matchId|           createdAt|    mapName|duration|mapSideLength|airplaneRoute_intercept|airplaneRoute_slope|          blueCircle|         whiteCircle|
+--------------------+--------------------+-----------+--------+-------------+-----------------------+-------------------+--------------------+--------------------+
|0ae4ee76-e0c2-4bb...|2022-05-23T06:06:32Z|Baltic_Main|    1619|       819200|      611591.9402010179|      -0.5766222533|[[0.0, 406387.5, ...|[[0.0, 0.0, 0.0, ...|
|168bf93d-38af-4aa...|2022-05-16T02:35:26Z|Baltic_Main|    1786|       819200|     1007058.4492530037|      -1.1085319537|[[0.0, 406387.5, ...|[[0.0, 0.0, 0.0, ...|
|3acf4fd9-716f-47c...|2022-05-28T01:25:02Z|Desert_Main|    1739|       819200|     -731698.7205816631|       3.2363852302|[[0.0, 408000.0, ...|[[0.0, 0.0, 0.0, ...|
|4f51f004-

In [6]:
playerLandingDF = spark.read.format("json").load("/sparkdata/PUBG/Matches1_PlayerLading.json").distinct()
playerLandingDF.show(10)

+--------------------+----------------+-------+--------------------+--------------------+
|             matchId|      playerName|ranking|                   x|                   y|
+--------------------+----------------+-------+--------------------+--------------------+
|b9337288-2cce-4f0...|J-SHOWMETHEMONEY|      6|{null, null, null...|{null, null, null...|
|3afe59b2-cff9-445...|       KR-LIZARD|      2|{null, null, null...|{null, null, null...|
|a2235a57-195d-470...|        BZG_OSAL|     12|{null, null, null...|{null, null, null...|
|a2235a57-195d-470...|       AMG_JIANT|      5|{null, null, null...|{null, null, null...|
|8a36f768-2c8b-4b7...|      KOR_JS3778|     12|{null, null, null...|{null, null, null...|
|a4bc5e6c-f1f1-452...|     Hyein2da-_-|     11|{null, null, null...|{null, null, null...|
|475c964d-c67d-4be...|  SwankyPantsGod|      2|{null, null, null...|{null, null, null...|
|92d7fcdd-9cf5-42f...|         DC_Rome|      6|{null, null, null...|{null, null, null...|
|6e5514ba-

In [7]:
def findPos(data):
    Pos=0
    for value in data:
        if not value == None:
            Pos = value
    return Pos

convertUDF = udf(lambda x:findPos(x),DoubleType())

playerLandingDF = playerLandingDF.select(col('matchId'),col('playerName'),col('ranking'),convertUDF(col('x')).alias('xPos')\
                                         ,convertUDF(col('y')).alias('yPos')).distinct()
playerLandingDF.printSchema()
playerLandingDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- xPos: double (nullable = true)
 |-- yPos: double (nullable = true)

+--------------------+----------------+-------+-------------+------------+
|             matchId|      playerName|ranking|         xPos|        yPos|
+--------------------+----------------+-------+-------------+------------+
|b9337288-2cce-4f0...|J-SHOWMETHEMONEY|      6|   458143.875| 626564.0625|
|3afe59b2-cff9-445...|       KR-LIZARD|      2|    399246.25|487224.71875|
|a2235a57-195d-470...|        BZG_OSAL|     12|   301561.125|430548.53125|
|a2235a57-195d-470...|       AMG_JIANT|      5|71759.5859375| 456179.3125|
|8a36f768-2c8b-4b7...|      KOR_JS3778|     12|  643030.8125|245492.71875|
|a4bc5e6c-f1f1-452...|     Hyein2da-_-|     11|212831.734375| 272995.9375|
|475c964d-c67d-4be...|  SwankyPantsGod|      2| 437845.71875|278797.65625|
|92d7fcdd-9cf5-42f...|         DC_Rome|      6|  58671

In [8]:
#팀단위로 변경
teamsLandingDF = playerLandingDF.groupBy('matchId','ranking').agg(avg('xPos').alias("teamXPos"),avg('yPos').alias("teamYPos"))
teamsLandingDF.printSchema()

teamsLandingDF = teamsLandingDF.join(matchInfoDF, teamsLandingDF['matchId']==matchInfoDF['matchId'],'inner')\
            .select(matchInfoDF['matchId'], matchInfoDF['mapName'], teamsLandingDF['ranking'] \
             ,teamsLandingDF['teamXPos'], teamsLandingDF['teamYPos'])

teamsLandingDF = teamsLandingDF.na.drop()
teamsLandingDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- teamXPos: double (nullable = true)
 |-- teamYPos: double (nullable = true)

+--------------------+-----------+-------+-------------+-----------------+
|             matchId|    mapName|ranking|     teamXPos|         teamYPos|
+--------------------+-----------+-------+-------------+-----------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|      7|595798.671875|        475962.75|
|0ae4ee76-e0c2-4bb...|Baltic_Main|      3|     449609.0|   313959.9140625|
|0ae4ee76-e0c2-4bb...|Baltic_Main|      4|158194.484375|   281945.4296875|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     15|586005.703125|   378576.3671875|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     10| 357920.03125|510593.1770833333|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     16|    520028.25|     419093.84375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     14| 670518.46875|    476747.078125|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     11|619373.140625|      605919.0625|
|0ae4ee76-e0c

In [9]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
#폴리곤정보 불러오기
ErangelPolygon = json.load(open("Erangel_LocalPos_Polygon.json"))
ErangelArea = dict()
for name,pos in ErangelPolygon.items():
    pos = (np.array(pos)*100).tolist()
    ErangelArea[name]=Polygon(pos)
    
MiramarPolygon = json.load(open("Miramar_LocalPos_Polygon.json"))
MiramarArea = dict()
for name,pos in MiramarPolygon.items():
    pos = (np.array(pos)*100).tolist()
    MiramarArea[name]=Polygon(pos)
#폴리곤정보로 명칭 변경
tempDF = teamsLandingDF.collect()
tempData = []
for data in tempDF:
    localName = 'another'
    mapName = data[1]
    if mapName == 'Baltic_Main':
        for name,polygon in ErangelArea.items():
            if polygon.contains(Point(data[3],data[4])):
                localName = name
    elif mapName == 'Desert_Main':
        for name,polygon in MiramarArea.items():
            if polygon.contains(Point(data[3],data[4])):
                localName = name
    tempData.append((data[0],data[1],data[2],localName))

landingPlaceDF = spark.createDataFrame(tempData,['matchId','mapName','ranking','landingPlace'])
landingPlaceDF.show(10)

#ErangelArea = sc.broadcast(dict())
#MiramarArea = sc.broadcast(dict())

+--------------------+-----------+-------+-------------------+
|             matchId|    mapName|ranking|       landingPlace|
+--------------------+-----------+-------+-------------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|      7|              Mylta|
|0ae4ee76-e0c2-4bb...|Baltic_Main|      3|MiddleBridgeVillage|
|0ae4ee76-e0c2-4bb...|Baltic_Main|      4|    Georgopol_south|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     15|            Shelter|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     10|      MiddleVillage|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     16|            another|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     14|            another|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     11|        Novorepnoye|
|0ae4ee76-e0c2-4bb...|Baltic_Main|      2|             School|
|0ae4ee76-e0c2-4bb...|Baltic_Main|     12|            another|
+--------------------+-----------+-------+-------------------+
only showing top 10 rows



In [10]:
#착지장소 장소별카운팅
landingPlaceDF.groupBy('mapName','landingPlace').agg(avg('ranking'),count('landingPlace')) \
    .select('*').orderBy('mapName').show(10)

+-----------+-------------------+-----------------+-------------------+
|    mapName|       landingPlace|     avg(ranking)|count(landingPlace)|
+-----------+-------------------+-----------------+-------------------+
|Baltic_Main|             School|9.615384615384615|               1066|
|Baltic_Main|              Mylta|8.822306238185256|                529|
|Baltic_Main|      ShootingRange|              8.8|                 35|
|Baltic_Main|         TopVillage|9.276018099547512|                221|
|Baltic_Main|            Lipovka| 8.58874458874459|                231|
|Baltic_Main|             Quarry|             7.92|                175|
|Baltic_Main|      MiddleVillage| 8.07750865051903|               1445|
|Baltic_Main|MiddleBridgeVillage|9.873096446700508|                197|
|Baltic_Main|   MilitaryBackYard|              7.4|                 25|
|Baltic_Main|     MilitaryBridge|9.585365853658537|                123|
+-----------+-------------------+-----------------+-------------

In [11]:
#착지장소 카운팅
#등수별 카운팅
temp1DF = landingPlaceDF.groupBy('mapName','LandingPlace','ranking').agg(count('ranking').alias('count'))
#장소별 카운팅
temp2DF = temp1DF.groupBy('mapName','landingPlace').agg(max('count').alias('max'),sum('count').alias('total'))

summary_landingPlaceDF = temp1DF.alias('SL').join(temp2DF.alias('PL'),\
                                        ((temp1DF['mapName']==temp2DF['mapName']) \
                                        &(temp1DF['landingPlace']==temp2DF['LandingPlace']) \
                                        &(temp1DF['count']==temp2DF['max'])),'inner') \
                                        .select('SL.mapName','SL.landingPlace','SL.ranking',\
                                        col('PL.max').alias('mode'),'PL.total').orderBy('mapName')

summary_landingPlaceDF.show(100,truncate = False)

+-----------+-------------------+-------+----+-----+
|mapName    |landingPlace       |ranking|mode|total|
+-----------+-------------------+-------+----+-----+
|Baltic_Main|MilitaryBackYard   |5      |5   |25   |
|Baltic_Main|Mylta              |7      |43  |529  |
|Baltic_Main|Georgopol_south    |7      |63  |888  |
|Baltic_Main|MyltaPower         |1      |29  |243  |
|Baltic_Main|TopVillage         |10     |18  |221  |
|Baltic_Main|Getka              |5      |21  |258  |
|Baltic_Main|Getka              |8      |21  |258  |
|Baltic_Main|Ferry Pier         |8      |14  |144  |
|Baltic_Main|School             |16     |102 |1066 |
|Baltic_Main|Farm               |1      |40  |406  |
|Baltic_Main|Novorepnoye        |3      |50  |507  |
|Baltic_Main|Ruins              |3      |8   |57   |
|Baltic_Main|RightSideVillage   |1      |26  |201  |
|Baltic_Main|Lipovka            |3      |20  |231  |
|Baltic_Main|MiddleVillage      |1      |115 |1445 |
|Baltic_Main|Severny            |9      |39  |

In [12]:
matchInfoDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- createdAt: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- mapSideLength: long (nullable = true)
 |-- airplaneRoute_intercept: double (nullable = true)
 |-- airplaneRoute_slope: double (nullable = true)
 |-- blueCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- whiteCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [13]:
#비행기 기울기 절편에 따른 동선명으로 반환
def airPlaneRouteTrans(data):
    mapSize = data[4]
    blockSize = mapSize/3
    slope = data[6]
    intercept = data[5]
    points = []
    points.append( (int((0-intercept)/slope),0) ) # tuple(x,y)
    points.append( (int(((mapSize/2)-intercept)/slope),mapSize/2) )
    points.append( (int((mapSize-intercept)/slope),mapSize) )
    blockPos = []
    for x, y in points:
        block=0
        if (x >= 0) and (x <= blockSize): #left
            block=1
        elif  x<=blockSize*2 : #middle
            block=2
        else: #right
            block=3
        if (y >= 0) and (y <= blockSize): #up
            block += 0
        elif y<=blockSize*2 : #middle
            block += 3
        else: #down
            block += 6
        blockPos.append(block)
        
    Route = "Don't Know"
    if blockPos[1] == 5:
        Route = "Middle"
    elif (blockPos[0],blockPos[2]) in [(2,8),(8,2),(4,6),(6,4),(1,9),(9,1),(3,7),(7,3)]:
        Route = "Middle"
    elif (blockPos[0],blockPos[2]) in [(1,3),(3,1),(7,9),(9,7),(1,7),(7,1),(3,9),(9,3)]:
        if (blockPos[0],blockPos[2]) in [(1,3),(3,1)]:
            Route = "UpSide"
        elif (blockPos[0],blockPos[2]) in [(7,9),(9,7)]:
            Route = "DownSide"
        elif (blockPos[0],blockPos[2]) in [(1,7),(7,1)]:
            Route = "LeftSide"
        elif (blockPos[0],blockPos[2]) in [(3,9),(9,3)]:
            Route = "RightSide"
    elif (blockPos[0] in [2,8]) or (blockPos[2] in [2,8]):
        if blockPos[1]==4:
            Route = "LeftSide_Middle"
        elif blockPos[1]==6:
            Route = "RightSide_Middle"
        else:
            Route = "Outer"
    return (data[0],data[1],data[2],data[3],data[4],Route,data[7],data[8])

tempRDD = matchInfoDF.rdd.map(airPlaneRouteTrans)
matchInfoDF = tempRDD.toDF(['matchId','createAt','mapName','duration','mapSize',\
                                'airplaneRoute','blueCircle','whiteCircle'])
matchInfoDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- createAt: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- mapSize: long (nullable = true)
 |-- airplaneRoute: string (nullable = true)
 |-- blueCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- whiteCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [14]:
matchInfoDF.select('matchId','mapName','airplaneRoute').show(10)

+--------------------+-----------+----------------+
|             matchId|    mapName|   airplaneRoute|
+--------------------+-----------+----------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|          Middle|
|168bf93d-38af-4aa...|Baltic_Main|          Middle|
|3acf4fd9-716f-47c...|Desert_Main|          Middle|
|4f51f004-d69d-4fe...|Desert_Main|          Middle|
|56901eea-f41e-46d...|Desert_Main|          Middle|
|6093b714-511c-4b4...|Desert_Main| LeftSide_Middle|
|afaef784-4d55-484...|Baltic_Main|          Middle|
|b9738d8c-18aa-4b1...|Baltic_Main|RightSide_Middle|
|c5137c90-9c11-45e...|Baltic_Main|RightSide_Middle|
|d3ce3c23-052e-416...|Baltic_Main|          Middle|
+--------------------+-----------+----------------+
only showing top 10 rows



In [15]:
#집계
matchInfoDF.groupBy('mapName','airplaneRoute').agg(count('airplaneRoute')).orderBy('mapName').show(100,truncate = False)

+-----------+----------------+--------------------+
|mapName    |airplaneRoute   |count(airplaneRoute)|
+-----------+----------------+--------------------+
|Baltic_Main|LeftSide_Middle |126                 |
|Baltic_Main|Middle          |688                 |
|Baltic_Main|RightSide_Middle|172                 |
|Baltic_Main|RightSide       |1                   |
|Desert_Main|Middle          |676                 |
|Desert_Main|LeftSide_Middle |146                 |
|Desert_Main|RightSide_Middle|143                 |
+-----------+----------------+--------------------+



In [16]:
landingPlaceDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- landingPlace: string (nullable = true)



In [17]:
#착지장소 데이터프레임에 해당 매치의 비행기 동선 추가
airplaneRoute_landingPlaceDF = landingPlaceDF.alias('LP').join(matchInfoDF.alias('MI'),["matchId","mapName"],'inner')\
        .select('LP.matchId','LP.mapName','LP.ranking','LP.LandingPlace','MI.airplaneRoute')
airplaneRoute_landingPlaceDF.printSchema()
airplaneRoute_landingPlaceDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- LandingPlace: string (nullable = true)
 |-- airplaneRoute: string (nullable = true)

+--------------------+-----------+-------+--------------+-------------+
|             matchId|    mapName|ranking|  LandingPlace|airplaneRoute|
+--------------------+-----------+-------+--------------+-------------+
|0f568f0d-6bed-478...|Desert_Main|     14|        impala|       Middle|
|0f568f0d-6bed-478...|Desert_Main|      2|     Chumacera|       Middle|
|0f568f0d-6bed-478...|Desert_Main|      4|        Pecado|       Middle|
|0f568f0d-6bed-478...|Desert_Main|     13|      ElAzahar|       Middle|
|0f568f0d-6bed-478...|Desert_Main|      6|MinasGenerales|       Middle|
|0f568f0d-6bed-478...|Desert_Main|     12|  MiddleVilage|       Middle|
|0f568f0d-6bed-478...|Desert_Main|      7|  UnderVillage|       Middle|
|0f568f0d-6bed-478...|Desert_Main|      3|      Military|       Middle

In [18]:
#등수 묶기 후 카운팅
def splitrank(data):
    if data<=3:
        return "HighRank"
    elif data>=10:
        return "MiddleRank"
    else:
        return "LowRank"
    return data

convertUDF = udf(lambda x:splitrank(x),StringType())

countByRank_landingPlaceDF = airplaneRoute_landingPlaceDF  \
            .select('mapName','airplaneRoute','LandingPlace',convertUDF(col('ranking')).alias('ranking'))  \
            .groupBy('mapName','airplaneRoute','LandingPlace','ranking').agg(count('ranking').alias('count'))

countByRank_landingPlaceDF.orderBy('mapName','airplaneRoute','LandingPlace').show(10)

+-----------+---------------+---------------+----------+-----+
|    mapName|  airplaneRoute|   LandingPlace|   ranking|count|
+-----------+---------------+---------------+----------+-----+
|Baltic_Main|LeftSide_Middle|       AquaCity|  HighRank|    3|
|Baltic_Main|LeftSide_Middle|       AquaCity|MiddleRank|    1|
|Baltic_Main|LeftSide_Middle|       AquaCity|   LowRank|    7|
|Baltic_Main|LeftSide_Middle|           Farm|MiddleRank|   14|
|Baltic_Main|LeftSide_Middle|           Farm|   LowRank|   13|
|Baltic_Main|LeftSide_Middle|           Farm|  HighRank|    9|
|Baltic_Main|LeftSide_Middle|     Ferry Pier|MiddleRank|    9|
|Baltic_Main|LeftSide_Middle|     Ferry Pier|  HighRank|    3|
|Baltic_Main|LeftSide_Middle|     Ferry Pier|   LowRank|    9|
|Baltic_Main|LeftSide_Middle|Georgopol_north|  HighRank|   21|
+-----------+---------------+---------------+----------+-----+
only showing top 10 rows



In [19]:
#총 카운팅
count_landingPlaceDF = countByRank_landingPlaceDF.groupBy('mapName','airplaneRoute','LandingPlace')  \
                                                  .agg(sum('count').alias('total'))

count_landingPlaceDF.orderBy('mapName','airplaneRoute','LandingPlace').show(10)

+-----------+---------------+-------------------+-----+
|    mapName|  airplaneRoute|       LandingPlace|total|
+-----------+---------------+-------------------+-----+
|Baltic_Main|LeftSide_Middle|           AquaCity|   11|
|Baltic_Main|LeftSide_Middle|               Farm|   36|
|Baltic_Main|LeftSide_Middle|         Ferry Pier|   21|
|Baltic_Main|LeftSide_Middle|    Georgopol_north|   87|
|Baltic_Main|LeftSide_Middle|    Georgopol_south|  248|
|Baltic_Main|LeftSide_Middle|              Getka|   44|
|Baltic_Main|LeftSide_Middle|           Kameshki|    2|
|Baltic_Main|LeftSide_Middle|            Lipovka|   19|
|Baltic_Main|LeftSide_Middle|            Mansion|    3|
|Baltic_Main|LeftSide_Middle|MiddleBridgeVillage|   14|
+-----------+---------------+-------------------+-----+
only showing top 10 rows



In [20]:
#비율로 변환
summary_landingPlaceDF = countByRank_landingPlaceDF.alias('SAL').join(count_landingPlaceDF.alias('PAL') \
             ,((countByRank_landingPlaceDF['mapName']==count_landingPlaceDF['mapName']) \
             &(countByRank_landingPlaceDF['LandingPlace']==count_landingPlaceDF['LandingPlace'])
             &(countByRank_landingPlaceDF['airplaneRoute']==count_landingPlaceDF['airplaneRoute'])),'inner')  \
             .select('SAL.mapName','SAL.airplaneRoute','SAL.LandingPlace','SAL.ranking' \
             ,'SAL.count','PAL.total').orderBy('mapName','ranking','airplaneRoute')

rankingOfLandingPlayerDF = summary_landingPlaceDF.select('mapName','airplaneRoute','LandingPlace','ranking' \
                ,expr(('round(count/total,3)')).alias('share')).orderBy('mapName','airplaneRoute','LandingPlace','ranking')

In [21]:
rankingOfLandingPlayerDF.select('*').filter((col('mapName')=='Baltic_Main')&(col('airplaneRoute')=='Middle'))\
    .orderBy('LandingPlace','ranking').show(100,truncate=False)

+-----------+-------------+-------------------+----------+-----+
|mapName    |airplaneRoute|LandingPlace       |ranking   |share|
+-----------+-------------+-------------------+----------+-----+
|Baltic_Main|Middle       |AquaCity           |HighRank  |0.213|
|Baltic_Main|Middle       |AquaCity           |LowRank   |0.319|
|Baltic_Main|Middle       |AquaCity           |MiddleRank|0.468|
|Baltic_Main|Middle       |Farm               |HighRank  |0.208|
|Baltic_Main|Middle       |Farm               |LowRank   |0.375|
|Baltic_Main|Middle       |Farm               |MiddleRank|0.416|
|Baltic_Main|Middle       |Ferry Pier         |HighRank  |0.204|
|Baltic_Main|Middle       |Ferry Pier         |LowRank   |0.439|
|Baltic_Main|Middle       |Ferry Pier         |MiddleRank|0.357|
|Baltic_Main|Middle       |Georgopol_north    |HighRank  |0.167|
|Baltic_Main|Middle       |Georgopol_north    |LowRank   |0.427|
|Baltic_Main|Middle       |Georgopol_north    |MiddleRank|0.406|
|Baltic_Main|Middle      

In [22]:
#플레이어 동선
playerPositionDF = spark.read.format('json').load("/sparkdata/PUBG/Matches1_PlayerPosition_Baltic_Main.json").distinct()
playerPositionDF.show(10)

+--------------------+----------------+--------------------+-------+
|             matchId|      playerName|            position|ranking|
+--------------------+----------------+--------------------+-------+
|21bf1ec8-4ee8-434...|          ATS_SY|[[5.481, 295967.6...|     14|
|b4fc5a91-a416-40c...|         NBG_xXx|[[1.649, 161841.7...|     14|
|b811ba90-7327-4d8...|       BoB_5king|[[2.754, 939804.0...|     16|
|e808b51a-881b-49a...|     Redsox_jjun|[[9.415, 187888.5...|      1|
|376564f4-b798-4b8...|       qjhwhwehq|[[8.677, 298904.6...|      5|
|75e31315-936e-48f...|         HB_SOSO|[[4.234, 914450.4...|     15|
|75e31315-936e-48f...|      WG_lliliil|[[5.939, 890111.4...|     10|
|e205544b-ae69-492...|OcO_Uncle_BongTV|[[2.052, 462802.5...|     12|
|e205544b-ae69-492...|  Kings_benjamin|[[2.286, 463071.6...|      7|
|3769da96-c4f0-4ae...|       bjRungRan|[[8.13, 795046.93...|     10|
+--------------------+----------------+--------------------+-------+
only showing top 10 rows



In [23]:
playerPositionDF.printSchema()
matchInfoDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- position: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- ranking: long (nullable = true)

root
 |-- matchId: string (nullable = true)
 |-- createAt: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- mapSize: long (nullable = true)
 |-- airplaneRoute: string (nullable = true)
 |-- blueCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- whiteCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [33]:
#자기장과 비교를 위해 조인연산
tempDF = playerPositionDF.alias('Pos').join(matchInfoDF.alias('Matches') \
             ,playerPositionDF['matchId']==matchInfoDF['matchId'],'inner')  \
             .select('Matches.matchId','Matches.mapName','Matches.duration','Matches.blueCircle','Matches.whiteCircle'\
             ,'Pos.playerName','Pos.ranking','Pos.position')

tempDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- blueCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- whiteCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- position: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [34]:
#총 이동거리로 변환
def DistanceData(data):
    import builtins
    round = getattr(builtins, "round")
    abs = getattr(builtins, "abs")
    Distance = 0
    position = data[7]
    liveTime = 0
    if len(position)!=0:
        liveTime = position[-1][0]
    if time==0: 
        time=10
    for idx, pos in enumerate(position[1:]):
        x2,y2 = pos[1],pos[2]
        x1,y1 = position[idx][1],position[idx][2]
        Distance += ((abs(x2-x1)**2)+(abs(y2-y1)**2))**(1/2) / 102.4
    Distance = round(Distance,10) / liveTime
    return (data[0],data[1],data[5],data[2],data[6], Distance, liveTime)

tempRDD = tempDF.rdd.map(DistanceData)
tempDF = tempRDD.toDF(['matchId','mapName','playerName','duration','ranking','moveDistance/sec','liveTime'])
tempDF = tempDF.select('mapName','ranking','moveDistance/sec',expr(('round(liveTime/duration,3)')).alias('liveTime'))
rankingOfMoveDistance = tempDF.groupBy('mapName','ranking').agg(mean('moveDistance/sec').alias('moveDistance/sec') \
                                                 ,mean('liveTime').alias('liveTime'))
rankingOfMoveDistance.orderBy('ranking').show(25)

+-----------+-------+------------------+------------------+
|    mapName|ranking|  moveDistance/sec|          liveTime|
+-----------+-------+------------------+------------------+
|Baltic_Main|      1| 7.410670372376336|0.9395665707559506|
|Baltic_Main|      2| 7.678145058748853|0.8778506370290053|
|Baltic_Main|      3|  8.10648809385848|0.8403799781778507|
|Baltic_Main|      4| 8.457241022837607|0.7869195813825394|
|Baltic_Main|      5| 8.987845454828944|0.7362867913990508|
|Baltic_Main|      6| 9.344184721627498|0.6976530727839139|
|Baltic_Main|      7| 9.595529453645053|0.6670470353477769|
|Baltic_Main|      8| 9.940691899027039|0.6398565365025468|
|Baltic_Main|      9| 10.50748348516057|0.6026341533180777|
|Baltic_Main|     10|10.714357192541113| 0.570880444064902|
|Baltic_Main|     11|11.244495755403852|0.5311543018335685|
|Baltic_Main|     12| 11.92006079487146|0.4831766517357223|
|Baltic_Main|     13|12.829385028235999|0.4273523627424202|
|Baltic_Main|     14|14.140742754989336|

In [26]:
#자기장과의 거리 값으로 변환
def DistancePlayerAndCircle(data):
    blueList = data[3]
    whiteList = data[4]
    
    timeLine = []
    InWhiteCircle = []
    DistanceByWhite = []
    DistanceByBlue = []
    
    weight = 0.7
    cnt=0
    for pos in data[7]:
        Ptime = pos[0]
        timeLine.append(Ptime)
        timecheck=0
        for point, Ctime in enumerate([tm[0] for tm in whiteList]):
            if Ptime < Ctime:
                timecheck = point
                break
                
        posX = pos[1]
        posY = pos[2]
        
        if ((cnt%18 == 0) & (cnt!=0)):
            weight *=0.7
        
        WcircleX = whiteList[point][1]
        WcircleY = whiteList[point][2]
        WcircleR = whiteList[point][4]
        
        import builtins
        abs = getattr(builtins, "abs") #내장함수abs와 spark.abs 덮어씌우기
        round = getattr(builtins, "round")

        if WcircleR > ((abs(WcircleX-posX)**2.0)+(abs(WcircleY-posY)**2.0))**(1/2):
            InWhiteCircle.append(True)
        else:
            InWhiteCircle.append(False)
        
        BcircleX = blueList[point][1]
        BcircleY = blueList[point][2]
        BcircleR = blueList[point][4]
        
        BDistance = (BcircleR - ((abs(BcircleX-posX)**2)+(abs(BcircleY-posY)**2))**(1/2))/BcircleR
        DistanceByBlue.append( round(BDistance * (1-weight),10) )
        
        if not WcircleR == 0:
            WDistance = 1- ((abs(WcircleX-posX)**2)+(abs(WcircleY-posY)**2))**(1/2) / BcircleR
            DistanceByWhite.append( round(WDistance * (1-weight),10) )
        cnt+=1
        
    return (data[0],data[1],data[2],data[5],data[6],timeLine,InWhiteCircle,DistanceByWhite,DistanceByBlue)

tempRDD = tempDF.rdd.map(DistancePlayerAndCircle)
distance_CircleAndPlayerDF = tempRDD.toDF(['matchId','mapName','duration','playerName','ranking','timeLine', \
                                                'InWhiteCircle','DistanceWhiteCircle','DistanceBlueCircle'])
distance_CircleAndPlayerDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- timeLine: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- InWhiteCircle: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- DistanceWhiteCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- DistanceBlueCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [27]:
distance_CircleAndPlayerDF.select('ranking','InWhiteCircle','DistanceWhiteCircle','DistanceBlueCircle').show(10)

+-------+--------------------+--------------------+--------------------+
|ranking|       InWhiteCircle| DistanceWhiteCircle|  DistanceBlueCircle|
+-------+--------------------+--------------------+--------------------+
|      9|[false, false, fa...|[0.1484710787, 0....|[0.0071996079, 0....|
|      9|[false, false, fa...|[0.1246222576, 0....|[0.0225475756, 0....|
|      5|[false, false, fa...|[0.2321889961, 0....|[0.0341807159, 0....|
|     11|[false, false, fa...|[0.2037010655, 0....|[0.0500183789, 0....|
|      8|[false, false, fa...|[0.1418095504, 0....|[0.0416051053, 0....|
|      3|[false, false, fa...|[0.1733993217, 0....|[0.0651100479, 0....|
|      4|[false, false, fa...|[0.0250214824, 0....|[0.0119029771, 0....|
|     10|[false, false, fa...|[0.1684187598, 0....|[0.076242477, 0.1...|
|     10|[false, false, fa...|[0.1014119903, 0....|[0.0755007952, 0....|
|      3|[false, false, fa...|[0.1753354324, 0....|[0.0500183789, 0....|
+-------+--------------------+--------------------+

In [28]:
#평균 거리로 변환
def listAVG(data):
    import builtins
    round = getattr(builtins, "round")
    sum = getattr(builtins, "sum")
    avg = 0
    if len(data)==0:
        avg = 0
    else:
        avg = round(sum(data)/len(data),5)
    return avg

convertUDF = udf(lambda x:listAVG(x),DoubleType())
tempDF = distance_CircleAndPlayerDF.select('mapName','ranking', \
                    convertUDF('DistanceBlueCircle').alias('DistanceBlueCircleAVG'), \
                    convertUDF('DistanceWhiteCircle').alias('DistanceWhiteCircleAVG'))

In [29]:
#등수별 평균거리
rankingOfDistanceCircleDF = tempDF.groupBy('mapName','ranking') \
    .agg(avg('DistanceBlueCircleAVG').alias('BlueDistRate') \
         ,avg('DistanceWhiteCircleAVG').alias('WhiteDistRate'))

In [30]:
rankingOfDistanceCircleDF.select('ranking','BlueDistRate').orderBy(desc('BlueDistRate')).show(50,truncate = False)

+-------+-------------------+
|ranking|BlueDistRate       |
+-------+-------------------+
|1      |0.35014364111953966|
|2      |0.3285417918134996 |
|3      |0.31460579650845616|
|4      |0.3135392564031948 |
|7      |0.3043853306727481 |
|5      |0.3028690589220888 |
|6      |0.29993080430472957|
|8      |0.2953243010752687 |
|9      |0.28658934210526327|
|20     |0.285085           |
|10     |0.2849956191289496 |
|14     |0.28496977326654366|
|13     |0.28461673586451786|
|11     |0.2841736276445697 |
|12     |0.2796297648376261 |
|15     |0.2760536541889484 |
|16     |0.25752455692652826|
|17     |0.23945175224123885|
|21     |0.233395           |
|19     |0.22995359999999995|
|18     |0.2238935329341317 |
+-------+-------------------+



In [31]:
rankingOfDistanceCircleDF.select('ranking','WhiteDistRate').orderBy(desc('WhiteDistRate')).show(50,truncate = False)

+-------+-------------------+
|ranking|WhiteDistRate      |
+-------+-------------------+
|1      |0.34572203766675375|
|2      |0.32160574410409304|
|3      |0.3075111429350791 |
|4      |0.30049497108234646|
|5      |0.28977465233175087|
|7      |0.2881866590649944 |
|6      |0.28234373654390943|
|8      |0.27461178268251285|
|14     |0.2645960717110466 |
|9      |0.2630505264663805 |
|10     |0.25993046114432106|
|11     |0.2589104315937941 |
|13     |0.25796411636164973|
|15     |0.25739638818135513|
|12     |0.25142468085106395|
|16     |0.2439183770076078 |
|20     |0.23125875         |
|18     |0.22879656957928796|
|17     |0.22330062028499578|
|19     |0.20838916666666668|
|21     |0.193025           |
+-------+-------------------+



In [32]:
sqlContext.clearCache()
#공격 로그
AttackDF = spark.read.format('json').load("/sparkdata/PUBG/Matches1_AttackerList.json")
AttackDF.printSchema()
AttackDF.show(10)

root
 |-- a_x: double (nullable = true)
 |-- a_y: double (nullable = true)
 |-- a_z: double (nullable = true)
 |-- attacker: string (nullable = true)
 |-- matchId: string (nullable = true)
 |-- time: double (nullable = true)
 |-- v_x: double (nullable = true)
 |-- v_y: double (nullable = true)
 |-- v_z: double (nullable = true)

+------------+-----------+-------------+--------+--------------------+------+------------+-----------+-------------+
|         a_x|        a_y|          a_z|attacker|             matchId|  time|         v_x|        v_y|          v_z|
+------------+-----------+-------------+--------+--------------------+------+------------+-----------+-------------+
|416962.09375|   668838.0|51.2099990845|  SMO1KE|ffff5561-157c-401...| 2.602|429306.59375|665495.5625|79.2099990845|
|416962.09375|   668838.0|51.2099990845|  SMO1KE|ffff5561-157c-401...| 3.804| 429277.6875|665477.1875|79.2099990845|
|416962.09375|   668838.0|51.2099990845|  SMO1KE|ffff5561-157c-401...|  6.31|428915.

In [33]:
#공격자, 피해자간의 거리로 변환
def DistanceAttacker(data):
    import builtins
    round = getattr(builtins, "round")
    abs = getattr(builtins, "abs")
    a_x = data[0]
    a_y = data[1]
    a_z = data[2]
    v_x = data[6]
    v_y = data[7]
    v_z = data[8]
    Distance_xy = ((abs(a_x-v_x)**2)+(abs(a_y-v_y)**2))**(1/2)
    Distance_xyz = ((Distance_xy**2)+(abs(a_z-v_z)**2))**(1/2)
    Distance_meter = Distance_xyz/102.4
    return (data[4],data[3],data[5],round(Distance_meter,3))

attackDistanceRDD = AttackDF.rdd.map(DistanceAttacker)
attackDistanceDF = attackDistanceRDD.toDF(['matchId','attacker','time','attackDistance'])
attackDistanceDF.printSchema()
attackDistanceDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- attacker: string (nullable = true)
 |-- time: double (nullable = true)
 |-- attackDistance: double (nullable = true)

+--------------------+--------+------+--------------+
|             matchId|attacker|  time|attackDistance|
+--------------------+--------+------+--------------+
|ffff5561-157c-401...|  SMO1KE| 2.602|       124.893|
|ffff5561-157c-401...|  SMO1KE| 3.804|       124.668|
|ffff5561-157c-401...|  SMO1KE|  6.31|       121.697|
|ffff5561-157c-401...|  SMO1KE|40.742|        10.062|
|ffff5561-157c-401...|  SMO1KE|40.742|        10.062|
|ffff5561-157c-401...|  SMO1KE|40.743|        10.062|
|ffff5561-157c-401...|  SMO1KE|40.743|        10.062|
|ffff5561-157c-401...|  SMO1KE|40.743|        10.062|
|ffff5561-157c-401...|  SMO1KE|44.949|         6.801|
|ffff5561-157c-401...|  SMO1KE| 44.95|         6.801|
+--------------------+--------+------+--------------+
only showing top 10 rows



In [34]:
distance_CircleAndPlayerDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- timeLine: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- InWhiteCircle: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- DistanceWhiteCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- DistanceBlueCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [35]:
#매치자기장 정보와 공격정보 묶기
attackDistanceAndMatchInfoDF = attackDistanceDF.distinct().alias('Atk').join(distance_CircleAndPlayerDF.alias('DCP') \
             ,((attackDistanceDF['matchId']==distance_CircleAndPlayerDF['matchId']) \
              & (attackDistanceDF['attacker']==distance_CircleAndPlayerDF['playerName'])),'inner')  \
             .select('DCP.matchId','DCP.mapName','DCP.duration','DCP.playerName','DCP.ranking','DCP.timeLine'\
             ,'DCP.DistanceBlueCircle','Atk.time','Atk.attackDistance')
attackDistanceAndMatchInfoDF.show(10)

+--------------------+-----------+--------+----------+-------+--------------------+--------------------+--------+--------------+
|             matchId|    mapName|duration|playerName|ranking|            timeLine|  DistanceBlueCircle|    time|attackDistance|
+--------------------+-----------+--------+----------+-------+--------------------+--------------------+--------+--------------+
|00e65f73-668f-49a...|Baltic_Main|    1928| 0w0____S2|     11|[7.743, 17.728, 2...|[0.0578773492, 0....| 717.515|       347.898|
|00e65f73-668f-49a...|Baltic_Main|    1928| 0w0____S2|     11|[7.743, 17.728, 2...|[0.0578773492, 0....| 543.031|       244.886|
|00e65f73-668f-49a...|Baltic_Main|    1928| 0w0____S2|     11|[7.743, 17.728, 2...|[0.0578773492, 0....|1095.048|        56.189|
|00e65f73-668f-49a...|Baltic_Main|    1928| 0w0____S2|     11|[7.743, 17.728, 2...|[0.0578773492, 0....| 524.816|         83.04|
|00e65f73-668f-49a...|Baltic_Main|    1928| 0w0____S2|     11|[7.743, 17.728, 2...|[0.0578773492,

In [36]:
#등수별 평균 교전 거리
rankingOfAttackDistanceDF = attackDistanceAndMatchInfoDF.groupBy('mapName','ranking').agg(avg('attackDistance') \
                                                .alias('avgDist')).select('mapName','ranking','avgDist')

In [37]:
rankingOfAttackDistanceDF.orderBy('ranking').show(25,truncate=False)

+-----------+-------+------------------+
|mapName    |ranking|avgDist           |
+-----------+-------+------------------+
|Baltic_Main|1      |122.67354347025613|
|Baltic_Main|2      |113.96805550552726|
|Baltic_Main|3      |114.00283081155432|
|Baltic_Main|4      |113.28867949751731|
|Baltic_Main|5      |109.36356887742798|
|Baltic_Main|6      |102.88543714238558|
|Baltic_Main|7      |104.9921046745133 |
|Baltic_Main|8      |99.75384359629513 |
|Baltic_Main|9      |91.52329258990198 |
|Baltic_Main|10     |90.03803301922929 |
|Baltic_Main|11     |82.06191728486647 |
|Baltic_Main|12     |71.67762468547811 |
|Baltic_Main|13     |64.61545128410113 |
|Baltic_Main|14     |53.57954296710248 |
|Baltic_Main|15     |38.78977381157672 |
|Baltic_Main|16     |30.81952732260162 |
|Baltic_Main|17     |25.134004651162787|
|Baltic_Main|18     |20.697947189097103|
|Baltic_Main|19     |25.270984293193717|
|Baltic_Main|20     |9.452666666666666 |
|Baltic_Main|21     |18.834            |
+-----------+---

In [38]:
#공격자에 대해서 공격정보 묶기 (GroupByKey 사용)
"""tempRDD = attackDistanceAndMatchInfoDF.select('mapName','playerName','ranking','time') \
    .rdd.map(lambda x : ((x[0],x[1],x[2]),x[3])).groupByKey().mapValues(list) \
    .map(lambda y : (y[0][0],y[0][1],y[0][2],y[1]))

attackGroupByDF = tempRDD.toDF(['mapName','playerName','ranking','times'])
attackGroupByDF.printSchema()
attackGroupByDF.show(10)"""

"tempRDD = attackDistanceAndMatchInfoDF.select('mapName','playerName','ranking','time')     .rdd.map(lambda x : ((x[0],x[1],x[2]),x[3])).groupByKey().mapValues(list)     .map(lambda y : (y[0][0],y[0][1],y[0][2],y[1]))\n\nattackGroupByDF = tempRDD.toDF(['mapName','playerName','ranking','times'])\nattackGroupByDF.printSchema()\nattackGroupByDF.show(10)"

In [39]:
#공격자에 대해서 공격정보 묶기 (ReduceByKey 사용)
tempRDD = attackDistanceAndMatchInfoDF.select('matchId','mapName','playerName','ranking','time') \
    .rdd.map(lambda row : ((row[0],row[1],row[2],row[3]),[row[4]])).reduceByKey(lambda x,y: x+y) \
        .map(lambda y : (y[0][0],y[0][1],y[0][2],y[0][3],y[1]))

attackGroupByDF = tempRDD.toDF(['matchId','mapName','playerName','ranking','times'])
attackGroupByDF.printSchema()
attackGroupByDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- times: array (nullable = true)
 |    |-- element: double (containsNull = true)

+--------------------+-----------+------------+-------+--------------------+
|             matchId|    mapName|  playerName|ranking|               times|
+--------------------+-----------+------------+-------+--------------------+
|16545297-863c-41b...|Baltic_Main|     14K-Tak|     11|[688.612, 935.469...|
|718c5db0-2b6d-44c...|Baltic_Main|     under24|     17|  [296.706, 299.076]|
|b4095108-5e6a-4a1...|Baltic_Main|     woo2-_-|      4|[1755.714, 1738.0...|
|e8c36172-471b-414...|Baltic_Main|     Msk_DOG|     10|[1123.966, 492.65...|
|49c8fd28-9cec-464...|Baltic_Main|  Roadkill_1|      6|[1576.55, 1362.69...|
|ecd63496-a7e5-422...|Baltic_Main|      E__SAK|     13|[926.453, 910.492...|
|0c178ae9-bd63-442...|Baltic_Main|aaaqqqwww___|      6|[1005

In [40]:
def combatCount(data):   
    times = data[4]
    cnt = 0
    combatStart = 0
    for time in times:
        if combatStart == 0:
            combatStart = time
            cnt += 1
        else:
            if time-combatStart >= 20:
                combatStart = time
                cnt +=1   
    return (data[0],data[1],data[2],data[3],cnt)


tempRDD = attackGroupByDF.rdd.map(combatCount)
rankingOfCombatCountDF = tempRDD.toDF(['matchId','mapName','playerName','ranking','combatCount'])
rankingOfCombatCountDF.printSchema()
rankingOfCombatCountDF.orderBy('ranking').show(30)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- combatCount: long (nullable = true)

+--------------------+-----------+--------------+-------+-----------+
|             matchId|    mapName|    playerName|ranking|combatCount|
+--------------------+-----------+--------------+-------+-----------+
|011f80b3-74b9-49d...|Baltic_Main|       shotgoa|      1|          4|
|80678a83-1311-47d...|Baltic_Main|   bigbozigirl|      1|          5|
|8c9ae95f-15fe-43d...|Baltic_Main|   TCN_lastman|      1|          1|
|fb1bdea2-b828-47e...|Baltic_Main|      AF_MXSIK|      1|          5|
|bf63b1a0-f0ad-49a...|Baltic_Main|      WWS_Dior|      1|          2|
|41d8d5f8-e58e-41d...|Baltic_Main|         VGLTN|      1|          4|
|de31bc7f-ac50-4a5...|Baltic_Main|     subal2-_-|      1|          2|
|03c8286a-1023-48c...|Baltic_Main|     themaia20|      1|          1|
|7c2b8dd5-3c5d-4bc...|Balt

In [41]:
rankingOfCombatCountDF.groupBy('mapName','ranking').agg(avg('combatCount').alias('combatCount')).orderBy('ranking').show(25)

+-----------+-------+------------------+
|    mapName|ranking|       combatCount|
+-----------+-------+------------------+
|Baltic_Main|      1|2.6555767657485685|
|Baltic_Main|      2|2.3670553935860057|
|Baltic_Main|      3| 2.228225568349572|
|Baltic_Main|      4| 2.116626065773447|
|Baltic_Main|      5| 2.011598746081505|
|Baltic_Main|      6|1.9102106969205834|
|Baltic_Main|      7|1.8986115595737811|
|Baltic_Main|      8|1.7776307081403044|
|Baltic_Main|      9|1.7032011134307585|
|Baltic_Main|     10| 1.688495575221239|
|Baltic_Main|     11| 1.608695652173913|
|Baltic_Main|     12|1.5229324665944384|
|Baltic_Main|     13|1.4856706152082537|
|Baltic_Main|     14|1.3623459096002988|
|Baltic_Main|     15| 1.313369397217929|
|Baltic_Main|     16| 1.262797074954296|
|Baltic_Main|     17|1.2099290780141845|
|Baltic_Main|     18|1.1818181818181819|
|Baltic_Main|     19|1.0731707317073171|
|Baltic_Main|     20|               1.0|
|Baltic_Main|     21|               1.0|
+-----------+---

In [42]:
attackDistanceAndMatchInfoDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- timeLine: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- DistanceBlueCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- time: double (nullable = true)
 |-- attackDistance: double (nullable = true)



In [43]:
#교전시 자기장 거리 정보
tempDF = attackGroupByDF.alias('AGB').join(distance_CircleAndPlayerDF.alias('DCP') \
             ,((attackGroupByDF['matchId']==distance_CircleAndPlayerDF['matchId']) \
              & (attackGroupByDF['mapName']==distance_CircleAndPlayerDF['mapName']) \
              & (attackGroupByDF['playerName']==distance_CircleAndPlayerDF['playerName']) \
              & (attackGroupByDF['ranking']==distance_CircleAndPlayerDF['ranking'])),'inner')  \
             .select('DCP.mapName','DCP.playerName','DCP.ranking','DCP.timeLine', 'DCP.DistanceBlueCircle','AGB.times')

tempDF.orderBy('playerName').show(10)

+-----------+----------+-------+--------------------+--------------------+--------------------+
|    mapName|playerName|ranking|            timeLine|  DistanceBlueCircle|               times|
+-----------+----------+-------+--------------------+--------------------+--------------------+
|Baltic_Main|    0-u-ya|      3|[0.608, 10.594, 2...|[0.0068924466, 0....|[1605.558, 848.79...|
|Baltic_Main|  000521-3|      1|[5.197, 15.184, 2...|[0.0397974237, 0....|[1822.203, 1826.0...|
|Baltic_Main|    000_vv|     11|[10.64, 20.627, 3...|[0.0781973219, 0....|           [740.908]|
|Baltic_Main|    000_vv|     12|[3.316, 13.301, 2...|[0.026767074, 0.1...|           [411.848]|
|Baltic_Main|    001_MN|      6|[9.274, 19.259, 2...|[0.0689831998, 0....|[1170.072, 1170.3...|
|Baltic_Main|    001_MN|      6|[2.901, 12.886, 2...|[0.0239310089, 0....|[149.248, 919.178...|
|Baltic_Main|    001_MN|     11|[9.895, 19.913, 2...|[0.0758593976, 0....|           [965.298]|
|Baltic_Main|    001_MN|      7|[0.872, 

In [44]:
def combatDistance(data): 
    import builtins
    sum = getattr(builtins, "sum")

    timeLine = data[3]
    length = len(timeLine)
    blue = data[4]
    attackTimes = data[5]
    attackDistance = []
    for time in attackTimes:
        for idx, stp in enumerate(timeLine):
            if idx < length:
                if (stp < time) | (timeLine[idx+1] >= time):
                    attackDistance.append(blue[idx])
                    break
            else:
                attackDistance.append(blue[idx])
                break
    DistAVG = sum(attackDistance) / len(attackDistance)
             
    return (data[0],data[1],data[2],DistAVG)


tempRDD = tempDF.rdd.map(combatDistance)
combatDistanceDF = tempRDD.toDF(['mapName','playerName','ranking','combatDistance'])
combatDistanceDF.printSchema()
combatDistanceDF.orderBy('ranking').show(10)

root
 |-- mapName: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- combatDistance: double (nullable = true)

+-----------+--------------+-------+--------------------+
|    mapName|    playerName|ranking|      combatDistance|
+-----------+--------------+-------+--------------------+
|Baltic_Main|    Y_DXNGYEOL|      1|0.011719624900000001|
|Baltic_Main|     RaveN5pro|      1| 0.06749004069999999|
|Baltic_Main|        JANG-U|      1| 0.06301335639999996|
|Baltic_Main|     VVS-Upppp|      1|0.013801692899999997|
|Baltic_Main|    hye_vely_-|      1|        0.0426803555|
|Baltic_Main|Gangsterparksm|      1| 0.05358327230000002|
|Baltic_Main|     SoN-_-No7|      1|        0.0093891422|
|Baltic_Main|     T_Bnb-Kue|      1|        0.0313594149|
|Baltic_Main|        RIP_GH|      1|        0.0389700646|
|Baltic_Main|      ex-Eyros|      1| 0.05215578349999997|
+-----------+--------------+-------+--------------------+
only showing top 1

In [45]:
rankingOfCombatDistanceDF = combatDistanceDF.groupBy('mapName','ranking').agg(avg('combatDistance').alias('combatDistance'))
rankingOfCombatDistanceDF.orderBy('ranking').show(25)

+-----------+-------+--------------------+
|    mapName|ranking|      combatDistance|
+-----------+-------+--------------------+
|Baltic_Main|      1| 0.04190316714425963|
|Baltic_Main|      2|0.042226765123381936|
|Baltic_Main|      3|  0.0418175074352229|
|Baltic_Main|      4| 0.04168572727375153|
|Baltic_Main|      5|0.042146549239341684|
|Baltic_Main|      6| 0.04171680111141006|
|Baltic_Main|      7|   0.041992515776784|
|Baltic_Main|      8| 0.04221387783272664|
|Baltic_Main|      9| 0.04090563618806541|
|Baltic_Main|     10| 0.04169922288718587|
|Baltic_Main|     11|0.042034149731254425|
|Baltic_Main|     12| 0.04211621810917298|
|Baltic_Main|     13| 0.04142524419197554|
|Baltic_Main|     14|0.041778389889802016|
|Baltic_Main|     15|0.041326630237132934|
|Baltic_Main|     16| 0.04216635152006399|
|Baltic_Main|     17| 0.04218386357489359|
|Baltic_Main|     18|0.039906402049431806|
|Baltic_Main|     19| 0.04396975666341464|
|Baltic_Main|     20|     0.0443754845125|
|Baltic_Mai

In [46]:
#차량탑승 로그
rideVehicleDF = spark.read.format('json').load("/sparkdata/PUBG/Matches1_RideVehicle.json")
rideVehicleDF.printSchema()
rideVehicleDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- player: string (nullable = true)
 |-- time: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)

+--------------------+--------------+--------------------+-------------+-------------+---------------+
|             matchId|        player|                time|            x|            y|              z|
+--------------------+--------------+--------------------+-------------+-------------+---------------+
|b9337288-2cce-4f0...|     K_JooHong|2022-05-14T14:18:...| 433253.84375| 467092.46875|2095.7080078125|
|b9337288-2cce-4f0...|What9Sangman_2|2022-05-14T14:18:...|     427737.0|    327429.25| 784.8502197266|
|b9337288-2cce-4f0...|    kim_seulki|2022-05-14T14:18:...| 408346.34375| 346234.53125| 208.9286804199|
|b9337288-2cce-4f0...|     K_JooHong|2022-05-14T14:18:...|  416194.9375|   476328.125|3009.9797363281|
|b9337288-2cce-4f0...|      BLOODC0W|2022-05-14T14:18:...| 27

In [47]:
rideVehicleDF.groupBy('matchId','player').agg(min('time').alias('firstRide')).orderBy('matchId').show(10,truncate=False)

+------------------------------------+----------------+------------------------+
|matchId                             |player          |firstRide               |
+------------------------------------+----------------+------------------------+
|0017e390-fdf6-4592-a1ce-1855258cb1a4|PLAT-_-RyeonGwan|2022-05-15T09:43:56.534Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|UNI_WooKi       |2022-05-15T09:44:23.002Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|Guoke-BaeMi     |2022-05-15T09:48:21.244Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|rjsemfaustkakd  |2022-05-15T09:46:38.794Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|LGT_WATSUB      |2022-05-15T09:49:15.104Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|KingPooh-_-     |2022-05-15T09:46:55.431Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|MMM_YuN2        |2022-05-15T09:51:07.010Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|Archi_S         |2022-05-15T09:43:40.925Z|
|0017e390-fdf6-4592-a1ce-1855258cb1a4|llI_H           |2022-05-15T09:52:38.927Z|
|0017e390-fdf6-4592-a1ce-185

In [48]:
matchInfoDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- createAt: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- mapSize: long (nullable = true)
 |-- airplaneRoute: string (nullable = true)
 |-- blueCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- whiteCircle: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [49]:
tempDF = rideVehicleDF.alias('RV').join(matchInfoDF.alias('MI') \
             ,rideVehicleDF['matchId']==matchInfoDF['matchId'],'inner')  \
             .select('RV.matchId','MI.mapName','MI.duration',col('RV.player').alias('playerName'),'MI.createAt'\
             ,'RV.time','RV.x','RV.y')
tempDF.show(5)

+--------------------+-----------+--------+----------+--------------------+--------------------+------------+------------+
|             matchId|    mapName|duration|playerName|            createAt|                time|           x|           y|
+--------------------+-----------+--------+----------+--------------------+--------------------+------------+------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|    AMF_6x|2022-05-23T06:06:32Z|2022-05-23T06:07:...| 271477.1875| 428035.9375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|   14K-nok|2022-05-23T06:06:32Z|2022-05-23T06:07:...|342503.78125|403012.28125|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|   14K-nok|2022-05-23T06:06:32Z|2022-05-23T06:07:...|342435.84375| 403038.0625|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|    AMF_6x|2022-05-23T06:06:32Z|2022-05-23T06:07:...|   290243.75| 446290.4375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|Limbinlord|2022-05-23T06:06:32Z|2022-05-23T06:07:...|448547.84375| 311692.9375|
+---------------

In [50]:
def timeSet(data): 
    import re
    import builtins
    sum = getattr(builtins, "sum")
    createAt=re.compile('T[0-9]{2}:[0-9]{2}:[0-9]{2}Z').findall(data[4])
    playTime=re.compile('T[0-9]{2}:[0-9]{2}:[0-9]{2}[.]').findall(data[5])
    (createHour,createMin,createSec) = re.findall("[0-9]{2}",createAt[0])
    (playHour,playMin,playSec) = re.findall("[0-9]{2}",playTime[0])
    Sec = int(playSec) - int(createSec)
    Min = int(playMin) - int(createMin)
    Hour = int(playHour) - int(createHour)
    #년도날짜요일에 대해서는 빼기 때문에 음수에 대해서 올림처리
    if Sec < 0:
        Sec = 60+Sec
        Min -=1
    if Min < 0:
        Min = 60+Min
        Hour -=1
    if Hour < 0:
        Hour = 24+Hour
        
    time = (Hour*3600)+(Min*60)+Sec
             
    return (data[0],data[1],data[2],data[3],time,data[6],data[7])


tempRDD = tempDF.rdd.map(timeSet)
rideVehicleTimeDF = tempRDD.toDF(['matchId','mapName','duration','playerName','rideTime','x','y'])
rideVehicleTimeDF.printSchema()
rideVehicleTimeDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- rideTime: long (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)

+--------------------+-----------+--------+----------------+--------+------------+------------+
|             matchId|    mapName|duration|      playerName|rideTime|           x|           y|
+--------------------+-----------+--------+----------------+--------+------------+------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|          AMF_6x|      70| 271477.1875| 428035.9375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|         14K-nok|      71|342503.78125|403012.28125|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|         14K-nok|      71|342435.84375| 403038.0625|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|          AMF_6x|      86|   290243.75| 446290.4375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|      Limbinlord|

In [51]:
distance_CircleAndPlayerDF.printSchema()
tempDF = distance_CircleAndPlayerDF.select('matchId','mapName','playerName','ranking')
tempDF.printSchema()
rideVehicleTimeDF.printSchema()

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- timeLine: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- InWhiteCircle: array (nullable = true)
 |    |-- element: boolean (containsNull = true)
 |-- DistanceWhiteCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- DistanceBlueCircle: array (nullable = true)
 |    |-- element: double (containsNull = true)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- playerName: string (nullable = true)
 |-- ranking: long (nullable = true)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- playerName: string (nullable = true)
 |-- rideTime: long (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = 

In [52]:
#등수 열을 얻기위해 다른 데이터 프레임과 조인연산
rideTempDF = rideVehicleTimeDF.alias('VT').join(tempDF.alias('tmp'), \
                    ((col('VT.matchId')==col('tmp.matchId')) \
                    & (col('VT.playerName')==col('tmp.playerName'))),'inner') \
                    .select('tmp.matchId','tmp.mapName','tmp.playerName' \
                            ,'tmp.ranking','VT.rideTime','VT.x','VT.y')
rideTempDF.show(10)

+--------------------+-----------+--------+----------------+--------+------------+------------+
|             matchId|    mapName|duration|      playerName|rideTime|           x|           y|
+--------------------+-----------+--------+----------------+--------+------------+------------+
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|          AMF_6x|      70| 271477.1875| 428035.9375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|         14K-nok|      71|342503.78125|403012.28125|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|         14K-nok|      71|342435.84375| 403038.0625|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|          AMF_6x|      86|   290243.75| 446290.4375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|      Limbinlord|      86|448547.84375| 311692.9375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|      Limbinlord|      86| 448604.6875| 311666.9375|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    1619|  Valentino_TANK|      87|450776.46875| 314672.3125|
|0ae4ee76-e0c2-4bb...|Baltic_Main|    16

In [53]:
rankingOfFirstRideVehicleDF = rideTempDF.groupBy('matchId','mapName','playerName','ranking') \
                        .agg(min('rideTime').alias('rideTime')).groupBy('mapName','ranking').agg(avg('rideTime').alias('rideTime'))
rankingOfFirstRideVehicleDF.filter(col('mapName')=='Baltic_Main').orderBy('ranking').show(25)

+-----------+-------+------------------+
|    mapName|ranking|          rideTime|
+-----------+-------+------------------+
|Baltic_Main|      1|275.76312880044225|
|Baltic_Main|      2|303.43414491053096|
|Baltic_Main|      3|309.04777351813624|
|Baltic_Main|      4|310.46486486486486|
|Baltic_Main|      5|317.63689559593473|
|Baltic_Main|      6| 327.7141515341265|
|Baltic_Main|      7| 322.3134138588684|
|Baltic_Main|      8| 325.1457349700976|
|Baltic_Main|      9|331.02439807383627|
|Baltic_Main|     10|  334.590792015454|
|Baltic_Main|     11| 322.4951861360719|
|Baltic_Main|     12|335.34328358208955|
|Baltic_Main|     13| 320.2096500169895|
|Baltic_Main|     14|303.99848082035703|
|Baltic_Main|     15| 280.9096098953378|
|Baltic_Main|     16|247.60850694444446|
|Baltic_Main|     17| 256.4807692307692|
|Baltic_Main|     18|180.48979591836735|
|Baltic_Main|     19|128.45454545454547|
+-----------+-------+------------------+



In [63]:
#차량을 타는 비율에 대해
rideTempDF = rideVehicleTimeDF.alias('VT').join(tempDF.alias('tmp'), \
                    ((col('VT.matchId')==col('tmp.matchId')) \
                    & (col('VT.playerName')==col('tmp.playerName'))),'rightouter') \
                    .select('tmp.matchId','tmp.mapName','tmp.playerName' \
                            ,'tmp.ranking','VT.rideTime','VT.x','VT.y').na.fill(0)

rideTempDF = rideTempDF.groupBy('matchId','mapName','ranking').agg(min('rideTime').alias('rideTime'))

In [64]:
rideTempDF.printSchema()
rideTempDF.show(10)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- rideTime: long (nullable = true)

+--------------------+-----------+-------+--------+
|             matchId|    mapName|ranking|rideTime|
+--------------------+-----------+-------+--------+
|a32bb497-01b4-4ef...|Baltic_Main|      2|     532|
|4749edfc-b0a9-434...|Baltic_Main|      1|     106|
|b2d20a91-52bd-4ca...|Baltic_Main|     11|     230|
|71ffe4b2-2245-40a...|Baltic_Main|     11|     777|
|b9738d8c-18aa-4b1...|Baltic_Main|      7|     217|
|f2f016fa-a98d-45e...|Baltic_Main|      4|     106|
|6a4b500f-bbac-493...|Baltic_Main|      4|      70|
|d6fc34e0-8359-49d...|Baltic_Main|      3|       0|
|de31bc7f-ac50-4a5...|Baltic_Main|     10|      73|
|3600bd64-e13b-469...|Baltic_Main|      8|      84|
+--------------------+-----------+-------+--------+
only showing top 10 rows



In [65]:
#0과 아닌것에 대해 구분
#공격자에 대해서 공격정보 묶기 (ReduceByKey 사용)
def splitZero(data):
    return bool(data)

convertUDF = udf(lambda x:splitZero(x),BooleanType())
rideTempDF = rideTempDF.select('matchId','mapName','ranking',convertUDF(col('rideTime')).alias('rideTime'))
rideTempDF.printSchema()
rideTempDF.show(100)

root
 |-- matchId: string (nullable = true)
 |-- mapName: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- rideTime: boolean (nullable = true)

+--------------------+-----------+-------+--------+
|             matchId|    mapName|ranking|rideTime|
+--------------------+-----------+-------+--------+
|a32bb497-01b4-4ef...|Baltic_Main|      2|    true|
|4749edfc-b0a9-434...|Baltic_Main|      1|    true|
|b2d20a91-52bd-4ca...|Baltic_Main|     11|    true|
|71ffe4b2-2245-40a...|Baltic_Main|     11|    true|
|b9738d8c-18aa-4b1...|Baltic_Main|      7|    true|
|f2f016fa-a98d-45e...|Baltic_Main|      4|    true|
|6a4b500f-bbac-493...|Baltic_Main|      4|    true|
|d6fc34e0-8359-49d...|Baltic_Main|      3|   false|
|de31bc7f-ac50-4a5...|Baltic_Main|     10|    true|
|3600bd64-e13b-469...|Baltic_Main|      8|    true|
|75e31315-936e-48f...|Baltic_Main|      7|   false|
|c6e0e2be-12c5-451...|Baltic_Main|     16|   false|
|8f3ca786-7cee-4d4...|Baltic_Main|     16|   false|
|ec56a

In [73]:
CountTempDF = rideTempDF.groupBy('mapName','ranking').agg(count('rideTime').alias('count'))
notRideTempDF = rideTempDF.filter(col('rideTime')==False).groupBy('mapName','ranking').agg(count('rideTime').alias('count'))
rideRateDF = CountTempDF.alias('TOTAL').join(notRideTempDF.alias('DONT'), \
                                   ((col('TOTAL.mapName')==col('DONT.mapName')) \
                                   &(col('TOTAL.ranking')==col('DONT.ranking'))),'inner') \
                                   .select('TOTAL.mapName','TOTAL.ranking' \
                                   ,expr(('1 - (DONT.count/TOTAL.count)')).alias('rideRate'))

rideRateDF.orderBy('ranking').show(25)

+-----------+-------+-------------------+
|    mapName|ranking|           rideRate|
+-----------+-------+-------------------+
|Baltic_Main|      1| 0.9207003089598352|
|Baltic_Main|      2| 0.8671472708547889|
|Baltic_Main|      3| 0.8712667353244078|
|Baltic_Main|      4| 0.8558187435633368|
|Baltic_Main|      5| 0.8362512873326468|
|Baltic_Main|      6|  0.831101956745623|
|Baltic_Main|      7| 0.8084449021627189|
|Baltic_Main|      8| 0.8084449021627189|
|Baltic_Main|      9| 0.7991761071060762|
|Baltic_Main|     10| 0.7894736842105263|
|Baltic_Main|     11| 0.7731958762886598|
|Baltic_Main|     12| 0.7254901960784313|
|Baltic_Main|     13| 0.6666666666666667|
|Baltic_Main|     14| 0.5254413291796469|
|Baltic_Main|     15| 0.3729338842975206|
|Baltic_Main|     16|0.16805721096543502|
|Baltic_Main|     17|0.09354838709677415|
|Baltic_Main|     18|0.06451612903225812|
|Baltic_Main|     19|0.05555555555555558|
|Baltic_Main|     20|                0.0|
|Baltic_Main|     21|             

In [74]:
rankingOfFirstRideVehicleDF = rankingOfFirstRideVehicleDF.alias('FR').join(rideRateDF.alias('rate'), \
                                            ((rankingOfFirstRideVehicleDF['mapName']==rideRateDF['mapName']) \
                                            &(rankingOfFirstRideVehicleDF['ranking']==rideRateDF['ranking'])),'rightouter')\
                                        .select('FR.mapName','FR.ranking','FR.rideTime','rate.rideRate')

rankingOfFirstRideVehicleDF.filter(col('mapName')=='Baltic_Main').orderBy('ranking').show(25)

+-----------+-------+------------------+-------------------+
|    mapName|ranking|          rideTime|           rideRate|
+-----------+-------+------------------+-------------------+
|Baltic_Main|      1|275.76312880044225| 0.9207003089598352|
|Baltic_Main|      2|303.43414491053096| 0.8671472708547889|
|Baltic_Main|      3|309.04777351813624| 0.8712667353244078|
|Baltic_Main|      4|310.46486486486486| 0.8558187435633368|
|Baltic_Main|      5|317.63689559593473| 0.8362512873326468|
|Baltic_Main|      6| 327.7141515341265|  0.831101956745623|
|Baltic_Main|      7| 322.3134138588684| 0.8084449021627189|
|Baltic_Main|      8| 325.1457349700976| 0.8084449021627189|
|Baltic_Main|      9|331.02439807383627| 0.7991761071060762|
|Baltic_Main|     10|  334.590792015454| 0.7894736842105263|
|Baltic_Main|     11| 322.4951861360719| 0.7731958762886598|
|Baltic_Main|     12|335.34328358208955| 0.7254901960784313|
|Baltic_Main|     13| 320.2096500169895| 0.6666666666666667|
|Baltic_Main|     14|303